<a href="https://colab.research.google.com/github/hannapalya/anomaly_detection_syndromic/blob/main/Farrington.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages("surveillance", repos = "https://cran.rstudio.com/")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘spatstat.data’, ‘spatstat.univar’, ‘spatstat.utils’, ‘deldir’, ‘polyclip’, ‘sp’, ‘polyCub’, ‘spatstat.geom’




In [ ]:
install.packages("zoo", repos = "https://cran.rstudio.com/")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
## ============================================
## Load pre-simulated datasets (totals/outbreaks)
## ============================================

data_dir <- "/signal_datasets_large"   # change if your CSVs live elsewhere

# Helper to load one signal's totals and outbreaks and bind to the expected names
.load_signal <- function(sig, dir = data_dir) {
  totals_fp    <- file.path(dir, sprintf("simulated_totals_sig%d.csv",    sig))
  outbreaks_fp <- file.path(dir, sprintf("simulated_outbreaks_sig%d.csv", sig))

  if (!file.exists(totals_fp))    stop("Missing file: ", totals_fp)
  if (!file.exists(outbreaks_fp)) stop("Missing file: ", outbreaks_fp)

  totals_df    <- read.csv(totals_fp,    check.names = FALSE)
  outbreaks_df <- read.csv(outbreaks_fp, check.names = FALSE)

  assign(sprintf("simulatedtotals%d",   sig), totals_df,    envir = .GlobalEnv)
  assign(sprintf("simulatedoutbreak%d", sig), outbreaks_df, envir = .GlobalEnv)
}

# Load signals 1..16
invisible(lapply(1:16, .load_signal))

# Load the seasonal-variant outbreak matrices for 5, 6, and 15
seas5_fp  <- file.path(data_dir, "simulated_seasonal_outbreaks_sig5.csv")
seas6_fp  <- file.path(data_dir, "simulated_seasonal_outbreaks_sig6.csv")
seas15_fp <- file.path(data_dir, "simulated_seasonal_outbreaks_sig15.csv")

if (!file.exists(seas5_fp))  stop("Missing file: ", seas5_fp)
if (!file.exists(seas6_fp))  stop("Missing file: ", seas6_fp)
if (!file.exists(seas15_fp)) stop("Missing file: ", seas15_fp)

simulatedzseasoutbreak5  <- read.csv(seas5_fp,  check.names = FALSE)
simulatedzseasoutbreak6  <- read.csv(seas6_fp,  check.names = FALSE)
simulatedzseasoutbreak15 <- read.csv(seas15_fp, check.names = FALSE)

# -------------------------------------------------
# Infer dimensions/params used later in the script
# -------------------------------------------------
N     <- nrow(simulatedtotals1)
nsim  <- ncol(simulatedtotals1)
days  <- 7
weeks <- N / days
years <- floor(N / (52 * days))  # should match how data were generated

# Quick sanity checks (optional; comment out if you like)
stopifnot(
  all(vapply(mget(paste0("simulatedtotals", 1:16)), nrow, integer(1)) == N),
  all(vapply(mget(paste0("simulatedoutbreak", 1:16)), nrow, integer(1)) == N),
  ncol(simulatedzseasoutbreak5)  == nsim,
  ncol(simulatedzseasoutbreak6)  == nsim,
  ncol(simulatedzseasoutbreak15) == nsim
)

cat(sprintf("Loaded datasets: N=%d rows, nsim=%d columns, days=%d, weeks=%.2f, years=%d\n",
            N, nsim, days, weeks, years))




#############################
#############################
# FIT FARRINGTON ALGO TO DATA
#############################
#############################

require(surveillance)
require(zoo)

#============================================
# Four functions that implement the algorithm
#============================================

algo.farrington=function (disProgObj, control = list(range = NULL, b = 3, w = 3,
                                                     reweight = TRUE, verbose = FALSE, alpha = 0.01, trend = TRUE,
                                                     limit54 = c(5, 4), powertrans = "2/3", fitFun = "algo.farrington.fitGLM"))
{
  observed <- disProgObj$observed

  freq <- disProgObj$freq
  #epochStr <- switch(as.character(freq), `12` = "1 month",
  #    `52` = "1 week", `365` = "1 day")
  if (is.null(control$range)) {
    control$range <- (freq * control$b - control$w):length(observed)
  }
  if (is.null(control$b)) {
    control$b = 5
  }
  if (is.null(control$w)) {
    control$w = 3
  }
  if (is.null(control$reweight)) {
    control$reweight = TRUE
  }
  if (is.null(control$verbose)) {
    control$verbose = FALSE
  }
  if (is.null(control$alpha)) {
    control$alpha = 0.01
  }
  if (is.null(control$trend)) {
    control$trend = TRUE
  }
  if (is.null(control$plot)) {
    control$plot = FALSE
  }
  if (is.null(control$limit54)) {
    control$limit54 = c(5, 4)
  }
  if (is.null(control$powertrans)) {
    control$powertrans = "2/3"
  }
  if (is.null(control$fitFun)) {
    control$fitFun = "algo.farrington.fitGLM"
  }
  #else {
  #    control$fitFun <- match.arg(control$fitFun, c("algo.farrington.fitGLM"))
  #}
  #if (is.null(disProgObj[["epochAsDate", exact = TRUE]])) {
  #    epochAsDate <- FALSE
  #}
  #else {
  #    epochAsDate <- disProgObj[["epochAsDate", exact = TRUE]]
  #}
  if (!((control$limit54[1] >= 0) & (control$limit54[2] > 0))) {
    stop("The limit54 arguments are out of bounds: cases >= 0 and period > 0.")
  }
  # alarmall gives the results for all data sets whereas alarm identifies the
  # ones where there are less than 5 reports in the last 4 weeks
  alarmall <- matrix(data = 0, nrow = length(control$range), ncol = 1)
  alarm<- matrix(data = 0, nrow = length(control$range), ncol = 1)
  upperbound <- matrix(data = 0, nrow = length(control$range), ncol = 1)
  trend <- matrix(data = 0, nrow = length(control$range), ncol = 1)
  pd <- matrix(data = 0, nrow = length(control$range), ncol = 2)

  n <- control$b * (2 * control$w + 1)

  # identify the leading zeroes in the data and cut them off

  #observed1=observed[observed!=NA]

  for (k in control$range) {
    #observed<-rollapply(data=observed[1:k], width=7, FUN=sum, ascending = F)
    #observed<-rollapply(data=a, width=7, FUN=sum, ascending = F)
    observed=observed[1:k]
    ob=rep(0,round(length(observed)/7))
    for(w in 0:(round(length(observed)/7)-1)){
      if((k-6*w-6-w)<=0){break}
      ob[w+1]=sum(observed[(k-6*w-w):(k-6*w-6-w)])
    }
    observed=ob[length(ob):1]
    kk=length(observed)
    for(z in 1:length(observed)){
      if(observed[z]==0){observed[z]=NA}
      else{break}
    }
    if (control$verbose) {
      cat("k=", k, "\n")
    }
    # If the remaining data is less than one year's worth,
    # do not fit a model and indicate that by reporting the value 1e+300
    if((kk-z)<(2*round(freq)+control$w+1)){
      upperbound[k - min(control$range) + 1] = 1e+300
      alarmall[k - min(control$range) + 1] = 1e+300
      alarm[k - min(control$range) + 1] = 1e+300
      trend[k - min(control$range) + 1] = 1e+300
    }
    else{
      #if (!epochAsDate) {
      # Assign level factors to the data (I have done that manually given
      # that the data can be missing...)
      seas1 <- NULL
      seas2 <- NULL
      for (i in control$b:0) {
        if(i==3){seas1=append(seas1, seq(kk - round(freq * i) -
                                           control$w, kk - round(freq * i) +control$w+1, by = 1))}
        else{seas1=append(seas1, seq(kk - round(freq * i) -
                                       control$w, kk - round(freq * i) +control$w, by = 1))}
        seas2=append(seas2, seq(kk - round(freq * i) -
                                  control$w-5, kk - round(freq * i) -control$w-1, by = 1))
      }
      #}
      seas3=seas2-5
      seas4=seas3-5
      seas5=seas4-5
      seas6=seas5-5
      seas7=seas6-5
      seas8=seas7-5
      seas9=seas8-5
      seas10=seas9-5
      seasgroup=rep(0,(length(observed)+control$w))
      seasgroup[seas1]=1
      seasgroup[seas2]=2
      seasgroup[seas3]=3
      seasgroup[seas4]=4
      seasgroup[seas5]=5
      seasgroup[seas6]=6
      seasgroup[seas7]=7
      seasgroup[seas8]=8
      seasgroup[seas9]=9
      seasgroup[seas10]=10
      if((kk-z)<(freq*control$b+control$w+1)){
        seasgroup=seasgroup[z:(kk-27)]
        seasgroup=as.factor(seasgroup)
        wtime= z:(kk-27)
      }
      else{
        seasgroup=seasgroup[(kk-control$b*freq-control$w-1):(kk-27)]
        seasgroup=as.factor(seasgroup)
        wtime= (kk-control$b*freq-control$w-1):(kk-27)
      }
      response <- observed[wtime]
      if (control$verbose) {
        print(response)
      }

      v=length(response[response>0])
      toosmall=(v<2) # indicator for when (stricktly) less than 2 weeks have non-zero counts

      # If stricktly less than 2 weeks have non-zero counts,
      # do not fit a model and indicate that by reporting the value 1e+400

      if(toosmall){
        upperbound[k - min(control$range) + 1] = 1e+400
        alarmall[k - min(control$range) + 1] = 1e+400
        alarm[k - min(control$range) + 1] = 1e+400
        trend[k - min(control$range) + 1] = 1e+400
      }
      else{
        oneyear=FALSE
        p=wtime[response>0]
        oneyear=((p[length(p)]-p[1])<52)
        # if all non-zero weeks are within one year, do not fit a trend.
        # This is the only case where we do not fit a trend.
        if(oneyear){
          model <- do.call(control$fitFun, args = list(response = response,
                                                       wtime = wtime, seasgroup=seasgroup,timeTrend = FALSE, reweight = control$reweight))
        }
        else{
          model <- do.call(control$fitFun, args = list(response = response,
                                                       wtime = wtime, seasgroup=seasgroup,timeTrend = control$trend, reweight = control$reweight))
        }
        if(is.null(model)){return(model)}

        doTrend <- control$trend
        if (model$T==1) {
          wp <- summary.glm(model)$coefficients["wtime", 4] # p-value after reweighting
          # In our new approach, we fit a trend always (unless all non-zero weeks are within one year).
          # For this reason, we consider the trend to be always significant (wp<=1).
          # If one wants to fit a trend only if significant, change this to, say, wp<=0.05.
          significant <- (wp <=1)
          mu0Hat <- predict.glm(model, data.frame(wtime = c(kk),seasgroup=factor(1)),type = "response")
          #atLeastThreeYears <- (control$b >= 3)
          # We remove the noExtrapolation condition
          #noExtrapolation <- mu0Hat <= max(response)
          if (!(significant)) {
            doTrend <- FALSE
            model <- do.call(control$fitFun, args = list(response = response,
                                                         wtime = wtime,seasgroup=seasgroup, timeTrend = FALSE, reweight = control$reweight))
          }
        }
        else {
          doTrend <- FALSE
        }
        if(model$phi<1){model$phi=1}
        pred <- predict.glm(model, data.frame(wtime = c(kk),seasgroup=factor(1)),
                            dispersion = model$phi, type = "response", se.fit = TRUE)
        # We use negative binomial quantiles to define the error structure rather
        # than the ones based on the transformed Poisson and the Anscombe residuals.
        if(model$phi==1){
          lu<-c(qpois(control$alpha,pred$fit),qpois(1-control$alpha,pred$fit))
          lu[2]=max(1,lu[2])
        }
        else{
          lu<-c(qnbinom(control$alpha,pred$fit/(model$phi-1),1/model$phi),qnbinom(1-control$alpha,pred$fit/(model$phi-1),1/model$phi))
          lu[2]=max(1,lu[2])
        }
        #if (control$plot) {
        #        data <- data.frame(wtime = seq(min(wtime), k, length = 1000))
        #        preds <- predict(model, data, type = "response",
        #            dispersion = model$phi)
        #        plot(c(wtime, k), c(response, observed[k]), ylim = range(c(observed[data$wtime],
        #            lu)), , xlab = "time", ylab = "No. infected",
        #            main = paste("Prediction at time t=", k, " with b=",
        #              control$b, ",w=", control$w, sep = ""), pch = c(rep(1,
        #              length(wtime)), 16))
        #        lines(data$wtime, preds, col = 1, pch = 2)
        #        lines(rep(k, 2), lu[1:2], col = 3, lty = 2)
        #    }

        enoughCases <- (sum(observed[(kk - control$limit54[2] + 1):kk]) >= control$limit54[1])

        X <- (observed[kk] - pred$fit)/(lu[2] - pred$fit)
        upperbound[k - min(control$range) + 1] <- lu[2]
        alarm[k - min(control$range) + 1] <- (X > 1)
        # if the last five weeks have less than 4 reports,
        # indicate that by returning the value 1e-300
        if(!enoughCases){alarm[k - min(control$range) + 1] = 1e-300}
        alarmall[k - min(control$range) + 1] <- (X > 1)
        trend[k - min(control$range) + 1] <- doTrend
      }
    }
    observed <- disProgObj$observed
  }
  control$name <- paste("farrington(", control$w, ",", 0, ",", control$b, ")", sep = "")
  control$data <- paste(deparse(substitute(disProgObj)))
  result <- list(alarm = alarm, alarmall = alarmall, trend = trend,
                 disProgObj = disProgObj, control = control,upperbound=upperbound)
  class(result) <- "survRes"
  return(result)
}

#=========================================

algo.farrington.fitGLM=function (response, wtime,seasgroup, timeTrend = TRUE, reweight = TRUE)
{
  theModel <- as.formula(ifelse(timeTrend, "response~seasgroup+wtime",
                                "response~seasgroup"))
  model <- glm(theModel, family = quasipoisson(link = "log"))
  # In our new approach, we fit a trend always (unless all non-zero weeks are within one year).
  # For this reason, we consider the trend always significant (p<=1).
  # If one wants to fit a trend only if significant, change this to, say, p<=0.05.
  if (model$converged){
    if (timeTrend) {
      p<- summary.glm(model)$coefficients["wtime", 4] # p-value before reweighting
      if(p<=1){T=1}
      else{
        T=0
        model <- glm(response ~ seasgroup, family = quasipoisson(link = "log"))
        if(!model$converged) {
          cat("Warning: No convergence without insignificant timeTrend.\n")
          print(cbind(response, wtime))
          return(NULL)
        }
      }
    }
    if(!timeTrend){T=0}
  }

  else {
    T=0
    if (timeTrend) {
      model <- glm(response ~ seasgroup, family = quasipoisson(link = "log"))
      cat("Warning: No convergence with timeTrend -- trying without.\n")
    }
    if (!model$converged) {
      cat("Warning: No convergence without timeTrend.\n")
      print(cbind(response, wtime))
      return(NULL)
    }
  }

  phi <- max(summary(model)$dispersion, 1)
  if (reweight) {
    s <- anscombe.residuals(model, phi)
    omega <- algo.farrington.assign.weights(s)
    theModel <- as.formula(ifelse(T==1, "response~seasgroup+wtime", "response~seasgroup"))
    model <- glm(theModel, family = quasipoisson(link = "log"),weights = omega)
    if (!model$converged) {

      if (T==1) {
        model <- glm(response ~ seasgroup, family = quasipoisson(link = "log"),weights = omega)
        cat("Warning: No convergence with weights and timeTrend -- trying without.\n")
      }
      if (!model$converged) {
        cat("Warning: No convergence with weights but without timeTrend.\n")
        print(cbind(response, wtime))
        return(NULL)
      }
      T=0
    }

    phi <- max(summary(model)$dispersion, 1)
  }
  model$phi <- phi
  model$T=T
  return(model)
}

#=========================================


algo.farrington.assign.weights=function (s)
{   # we use the cut off point s>2.58 as a compromise between s>2 and s>3
  gamma <- length(s)/(sum((s^(-2))^(s > 2.58)))
  omega <- numeric(length(s))
  omega[s > 2.58] <- gamma * (s[s > 2.58]^(-2))
  omega[s <= 2.58] <- gamma
  return(omega)
}


#=========================================


anscombe.residuals=function (m, phi)
{
  y <- m$y
  mu <- fitted.values(m)
  a <- 3/2 * (y^(2/3) * mu^(-1/6) - mu^(1/2))
  a <- a/sqrt(phi * (1 - hatvalues(m)))
  return(a)
}


#==========================================



#=============================
# Define the alarm data frames
#=============================

days=7

nsim=100

alarm1=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm2=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm3=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm4=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm5=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm6=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm7=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm8=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm9=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm10=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm11=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm12=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm13=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm14=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm15=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarm16=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))




alarmall1=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall2=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall3=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall4=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall5=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall6=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall7=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall8=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall9=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall10=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall11=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall12=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall13=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall14=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall15=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))
alarmall16=data.frame(array(rep(0,nsim*49*days),dim=c(49*days,nsim)))



#========================================
#Implement the algorithm to data by days
#========================================

#require(surveillance)

b=5
weeks=N/7

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals1[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals1)-49*days+1):nrow(simulatedtotals1),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm1[,i]=a.farrington$alarm[,1]
  alarmall1[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals2[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals2)-49*days+1):nrow(simulatedtotals2),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm2[,i]=a.farrington$alarm[,1]
  alarmall2[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals3[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals3)-49*days+1):nrow(simulatedtotals3),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm3[,i]=a.farrington$alarm[,1]
  alarmall3[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals4[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals4)-49*days+1):nrow(simulatedtotals4),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm4[,i]=a.farrington$alarm[,1]
  alarmall4[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals5[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals5)-49*days+1):nrow(simulatedtotals5),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm5[,i]=a.farrington$alarm[,1]
  alarmall5[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals6[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals6)-49*days+1):nrow(simulatedtotals6),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm6[,i]=a.farrington$alarm[,1]
  alarmall6[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals7[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals7)-49*days+1):nrow(simulatedtotals7),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm7[,i]=a.farrington$alarm[,1]
  alarmall7[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals8[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals8)-49*days+1):nrow(simulatedtotals8),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm8[,i]=a.farrington$alarm[,1]
  alarmall8[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals9[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals9)-49*days+1):nrow(simulatedtotals9),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm9[,i]=a.farrington$alarm[,1]
  alarmall9[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals10[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals10)-49*days+1):nrow(simulatedtotals10),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm10[,i]=a.farrington$alarm[,1]
  alarmall10[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals11[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals11)-49*days+1):nrow(simulatedtotals11),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm11[,i]=a.farrington$alarm[,1]
  alarmall11[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals12[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals12)-49*days+1):nrow(simulatedtotals12),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm12[,i]=a.farrington$alarm[,1]
  alarmall12[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals13[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals13)-49*days+1):nrow(simulatedtotals13),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm13[,i]=a.farrington$alarm[,1]
  alarmall13[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals14[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals14)-49*days+1):nrow(simulatedtotals14),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm14[,i]=a.farrington$alarm[,1]
  alarmall14[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals15[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals15)-49*days+1):nrow(simulatedtotals15),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm15[,i]=a.farrington$alarm[,1]
  alarmall15[,i]=a.farrington$alarmall[,1]
}

for(i in 1:nsim){
  a.disProg=create.disProg(week=weeks,observed=simulatedtotals16[,i],freq=52.18)
  cntrl <- list(range=(nrow(simulatedtotals16)-49*days+1):nrow(simulatedtotals16),w = 3, b = 5, alpha = 0.01,trend=TRUE, fitFun="algo.farrington.fitGLM")
  a.farrington<- algo.farrington(a.disProg, control = cntrl)
  alarm16[,i]=a.farrington$alarm[,1]
  alarmall16[,i]=a.farrington$alarmall[,1]
}


# save.image("//penelope/MCSUsers/Staff/an3936/Documents/own research/RAMMIE/R/outx2.Rdata")

#====================================
#====================================
#Summary
#====================================
#====================================
## ───────────────────────────────────────────────────────────────
## 1.  Parameters & convenience objects
## ───────────────────────────────────────────────────────────────
nsim      <- 1000               # already defined earlier
days      <- 7                 # every series is 7-day service in your manual code
years     <- 7                 # needed for timeliness
idx_range <- 2206:2548         # rows you used for the denominators
seas_ids  <- c(5, 6, 15)       # those with seasonal “zseas” variants

## 2.  Collect all alarm/outbreak matrices into lists
alarmall_list     <- mget(paste0("alarmall",            1:16))
outbreak_list     <- mget(paste0("simulatedoutbreak",    1:16))
zseas_outbreak_ls <- list(`5`  = simulatedzseasoutbreak5,
                          `6`  = simulatedzseasoutbreak6,
                          `15` = simulatedzseasoutbreak15)

## ───────────────────────────────────────────────────────────────
## 3.  Helper functions  ― exactly the same indexing as loops
## ───────────────────────────────────────────────────────────────
compute_fpr <- function(A, O, idx) {
  n  <- nrow(A)
  FP <- sum(A == 1 & tail(O, n) == 0)
  N0 <- sum(O[idx, ] == 0)
  if (N0 == 0) NA else FP / N0
}

compute_specificity <- function(A, O, idx) {
  n  <- nrow(A)
  TN <- sum(A == 0 & tail(O, n) == 0)
  N0 <- sum(O[idx, ] == 0)
  if (N0 == 0) NA else TN / N0
}

compute_sensitivity <- function(A, O) {
  n  <- nrow(A)
  TP <- sum(A == 1 & tail(O, n) > 0)
  P  <- sum(O > 0)
  if (P == 0) NA else TP / P
}

compute_pod <- function(A, O) {            # “power of detection”
  n <- nrow(A)
  mean(colSums(A == 1 & tail(O, n) > 0) > 0)
}

compute_timeliness <- function(A, O, days, years) {
  nsim   <- ncol(A)
  n_alarm <- nrow(A)
  n_out   <- nrow(O)
  miss    <- 0
  score   <- 0

  for (j in seq_len(nsim)) {
    ## r2 = last day of the outbreak window
    r2 <- max(which(O[(52*days*(years-1)+3*days+1):(52*days*years), j] > 0)) +
      52*days*(years-1)+3*days
    ## r1 = first day of the outbreak window
    r1 <- min(which(O[(52*days*(years-1)+3*days+1):(52*days*years), j] > 0)) +
      52*days*(years-1)+3*days

    if (length(r1) == 0 || length(r2) == 0) { miss <- miss + 1; next }

    ## first alarm that coincides with any outbreak-day in the tail
    alarm_hit <- which(A[, j] == 1 & tail(O[, j], n_alarm) > 0)
    if (length(alarm_hit)) {
      obs_idx <- n_out - n_alarm + alarm_hit[1]
      score   <- score + (obs_idx - r1) / (r2 - r1 + 1)
    } else {
      miss <- miss + 1
    }
  }
  (score + miss) / nsim
}

## ───────────────────────────────────────────────────────────────
## 4.  Core metric loops  (16 regular + 3 seasonal)
## ───────────────────────────────────────────────────────────────
fpr  <- pod <- sensitivity <- specificity <- timeliness <- numeric(16)
fprS <- podS <- sensitivityS <- specificityS <- timelinessS <- numeric(3)

for (i in 1:16) {
  cat(sprintf("i=%d: alarmall %dx%d, outbreak %dx%d\n",
              i,
              nrow(alarmall_list[[i]]), ncol(alarmall_list[[i]]),
              nrow(outbreak_list[[i]]), ncol(outbreak_list[[i]])))
}

for (i in 1:16) {
  A <- alarmall_list[[i]]
  O <- outbreak_list[[i]]

  fpr[i]         <- compute_fpr        (A, O, idx_range)
  specificity[i] <- compute_specificity(A, O, idx_range)
  sensitivity[i] <- compute_sensitivity(A, O)
  pod[i]         <- compute_pod        (A, O)
  timeliness[i]  <- compute_timeliness (A, O, days, years)
}

for (k in seq_along(seas_ids)) {
  i <- seas_ids[k]
  A <- alarmall_list[[i]]
  O <- zseas_outbreak_ls[[as.character(i)]]

  fprS[k]         <- compute_fpr        (A, O, idx_range)
  specificityS[k] <- compute_specificity(A, O, idx_range)
  sensitivityS[k] <- compute_sensitivity(A, O)
  podS[k]         <- compute_pod        (A, O)
  timelinessS[k]  <- compute_timeliness (A, O, days, years)
}

## ───────────────────────────────────────────────────────────────
## 5.  Assemble the two summary data frames
## ───────────────────────────────────────────────────────────────
Summary1  <- data.frame(
  fpr, pod, sensitivity, specificity, timeliness,
  row.names = paste0("sigid", 1:16)
)

Summary2 <- data.frame(
  fpr  = fprS,  pod  = podS,
  sensitivity = sensitivityS,
  specificity = specificityS,
  timeliness  = timelinessS,
  row.names = paste0("sigid", seas_ids)
)

Summary1
Summary2

Loaded datasets: N=2548 rows, nsim=1000 columns, days=7, weeks=364.00, years=7


A streamkimeneten csak az utolsó 5000 sor látható.
Warning message in create.disProg(week = weeks, observed = simulatedtotals1[, i], :
“'create.disProg' is deprecated.
Use 'sts' instead.
See help("Deprecated")”
Warning message in create.disProg(week = weeks, observed = simulatedtotals1[, i], :
“'create.disProg' is deprecated.
Use 'sts' instead.
See help("Deprecated")”
Warning message in create.disProg(week = weeks, observed = simulatedtotals1[, i], :
“'create.disProg' is deprecated.
Use 'sts' instead.
See help("Deprecated")”
Warning message in create.disProg(week = weeks, observed = simulatedtotals1[, i], :
“'create.disProg' is deprecated.
Use 'sts' instead.
See help("Deprecated")”
Warning message in create.disProg(week = weeks, observed = simulatedtotals1[, i], :
“'create.disProg' is deprecated.
Use 'sts' instead.
See help("Deprecated")”
Warning message in create.disProg(week = weeks, observed = simulatedtotals1[, i], :
“'create.disProg' is deprecated.
Use 'sts' instead.
See help("Dep

ERROR: Error: object 's' not found
